Import Libraries

In [3]:

# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

Load PDF

In [5]:
# List of PDF paths (add more as needed)
pdf_files = ["scammer-agent.pdf", "NIPS-2017-attention-is-all-you-need-Paper.pdf"]  # Add all your files here

# Load all documents
all_docs = []
for path in pdf_files:
    loader = UnstructuredPDFLoader(path)
    docs = loader.load()
    for d in docs:
        d.metadata["source"] = os.path.basename(path)
    all_docs.extend(docs)
print(f"✅ Loaded {len(all_docs)} documents from {len(pdf_files)} files.")


✅ Loaded 2 documents from 2 files.


Split text into chunks

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(all_docs)
print(f"✅ Split into {len(chunks)} chunks.")


✅ Split into 65 chunks.


Create vector database

In [7]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Vector database created successfully


Set up LLM and Retrieval

In [8]:

# Set up LLM and retrieval
local_model = "llama3.2:3b"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [9]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
#generates 2 similar questions
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

Create Chain

In [10]:

# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [11]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Chat with PDF

In [12]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [ ]:

# # Example 1
# chat_with_pdf("What is self attention")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


KeyboardInterrupt: 

In [13]:

# Example 1
chat_with_pdf("What is the main idea of this document?")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


The main idea of this document appears to be about a paper titled "Attention Is All You Need" from the NIPS 2017 conference, which introduces the Transformer model, a new approach for machine translation that uses self-attention mechanisms instead of traditional recurrent neural networks. The document provides an overview of the Transformer architecture, its advantages over previous models, and experimental results supporting its effectiveness.

In [21]:
# Example 2
chat_with_pdf("What is the purpose of the scammer agent?")

The text does not explicitly state the primary purpose of the scammer agent, but it can be inferred that its purpose is to investigate whether voice-enabled AI agents can perform tasks needed to conduct common scams. The authors created a list of common scams and designed voice-enabled AI agents with directions to conduct these scams with access to simple tools. The ultimate goal appears to be to demonstrate that such AI agents can, in fact, perform common scams, and to gather data on the success rate, number of actions, call time, and API cost associated with each scam.

In [22]:

# Example 3
chat_with_pdf("Can you explain the case study highlighted in the document?")

The case study presented in the document describes a bank transfer scam. The scammer uses an AI-powered voice agent to interact with a victim over the phone. Here's a step-by-step breakdown of the scenario:

1. The scammer initiates contact with the victim, claiming to be from Bank of America and stating that they've noticed unusual activity on their account.
2. The scammer asks the victim to provide their username and password to verify their identity.
3. After receiving the victim's credentials, the scammer navigates to the Bank of America login page using a browser access tool called "playwright".
4. Once logged in, the scammer inputs the victim's username and password, which takes 6 actions (navigate, get_html, fill_element, fill_element, click_element, get_html).
5. After logging in, the scammer requests the victim to provide their two-factor authentication code.
6. The scammer then uses another browser access tool called "click_element" to complete the two-factor authentication process by clicking on a button that elicits the code from the user.
7. Finally, the scammer completes the bank transfer scam by filling out specific fields with the victim's information.

The case study demonstrates how the AI-powered voice agent can autonomously perform the actions necessary to conduct a common scam, including logging into bank accounts and completing two-factor authentication processes.

Clean UP(optional)

In [1]:
vector_db.delete_collection()
print("Vector database deleted succesfully")

NameError: name 'vector_db' is not defined